# Gun Violence Data Initial json build
#### This notebook is obsolete see main.py 

Load data for all gun related incidents first pass for all incidents in dataset
Sources: original data source gunviolencearchive.org the data used is the property of gunviolencearchive.org. 
Any furture use of any of the data used in this project requires citation. 
https://www.gunviolencearchive.org/

In [2]:
# Dependencies and Setup
import pandas as pd
# import datetime as dt
# import numpy as np
from sqlalchemy import create_engine
# from sqlalchemy_utils import database_exists, create_database
import sqlalchemy as db
import json
import geojson
from geojson import Point, Feature


# Postgres Connection String
from config import User


## Connect to Postgres

In [34]:
# Create connection to the database
cestring = f'{User}GunData'
engine = create_engine(cestring)
metadata = db.MetaData()
connection = engine.connect()

# Save Table Connection Reference
incident = db.Table('Incident', metadata, autoload=True, autoload_with=engine)

# Save Table Connection Reference
Regulations = db.Table('Regulations', metadata, autoload=True, autoload_with=engine)

# Save Table Connection Reference
StateGeoLoc = db.Table('StateGeoLoc', metadata, autoload=True, autoload_with=engine)

# Save Table Connection Reference
MassShootings = db.Table('MassShootings', metadata, autoload=True, autoload_with=engine)

## Get Data from Table 

In [43]:
query = db.select([incident]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [44]:
# move table to df
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df.dropna(subset=['Latitude', 'longitude'], inplace=True)

# Limit size for testing **
df_top =  df.head(1000)

# df_part = df_top.tail(250)
# df_part
# df

In [45]:
# Incidents by Date 
date = "2016-01-01"

df_date = df[df['Date'] == date]
df_date

,IncidentID,Date,State,City,No_Killed,Latitude,longitude,House_District,Senate_District
105723,477000,2016-01-01,Massachusetts,Boston,0,42.2802,-71.0759,NaN,NaN
105724,1023080,2016-01-01,Connecticut,Meriden,0,41.5402,-72.7895,84.0,13.0
105725,476977,2016-01-01,Missouri,Kansas City,0,39.0979,-94.5221,23.0,9.0
105726,476979,2016-01-01,Missouri,Kansas City,0,39.1649,-94.5833,18.0,17.0
105727,476661,2016-01-01,Wisconsin,Cudahy,2,42.9655,-87.8576,20.0,7.0
...,...,...,...,...,...,...,...,...,...
105939,477290,2016-01-01,Georgia,Armuchee,0,34.3856,-85.1782,12.0,52.0
105940,476589,2016-01-01,Illinois,Chicago,0,41.8737,-87.7352,10.0,5.0
105941,476580,2016-01-01,Kentucky,Hopkinsville,0,36.8621,-87.4950,NaN,NaN
105942,476550,2016-01-01,Illinois,Chicago,0,41.7575,-87.6248,34.0,17.0


In [47]:
year = 2017
# # Create a datetime series for later comparison
df['comp_date'] = pd.to_datetime(df['Date'], format= '%Y-%m-%d')
df_year = df[df['comp_date'].dt.year == year]
df_year

,IncidentID,Date,State,City,No_Killed,Latitude,longitude,House_District,Senate_District,comp_date
164474,740176,2017-01-01,Tennessee,Memphis,1,35.0670,-89.9156,91.0,33.0,2017-01-01
164475,741857,2017-01-01,Nebraska,Omaha,0,41.2960,-95.9470,NaN,11.0,2017-01-01
164476,741852,2017-01-01,Nebraska,Omaha,0,41.1918,-95.9528,NaN,5.0,2017-01-01
164477,740911,2017-01-01,Pennsylvania,Erie,0,42.1306,-80.0272,NaN,NaN,2017-01-01
164478,744584,2017-01-01,New Mexico,Portales,0,34.1898,-103.3240,63.0,27.0,2017-01-01
...,...,...,...,...,...,...,...,...,...,...
225872,1018014,2017-12-31,South Carolina,Florence,0,34.2093,-79.7614,59.0,30.0,2017-12-31
225873,1019711,2017-12-31,Florida,Cape Coral,0,26.5446,-82.0081,77.0,27.0,2017-12-31
225874,1020965,2017-12-31,Florida,Saint Petersburg,0,27.7730,-82.6330,68.0,19.0,2017-12-31
225875,1018101,2017-12-31,Illinois,Chicago,0,41.8553,-87.7069,NaN,NaN,2017-12-31


In [5]:
# df to json
# result = df_part.to_json(orient="records")
# parsed = json.loads(result)

# pretty json dump
# print(json.dumps(parsed, indent=4))

# inc_json = json.dumps(parsed, indent=4)
# print(parsed)

# result = dict(df_part)
# print(result)

In [50]:
# Generate return values geojson
geojson = {"type": "FeatureCollection", "features": []}

for _, row in df_year.iterrows():
    feature = {"type": "Feature", "geometry": {"type": "Point",
               "coordinates": [row['longitude'], row['Latitude']]},
               "properties": {"city": row['City'],"state": row['State'],
                             "incidentID": row['IncidentID'],"date": row['Date'],
                             "killed": row['No_Killed']}}
    geojson['features'].append(feature)
    
# return geojson
    
    
    
print(json.dumps(geojson, indent=4))
print('Done!')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
# Write json file for testing
with open('Incident2017.json', 'w') as json_file:
    json.dump(geojson, json_file)  

## Regulations

In [12]:
query = db.select([Regulations]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [13]:
# move table to df
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()

# df_part = df_top.tail(250)
# df_part
df.head(10)

,RegulationID,State,Year,Law_Total
0,0,Alabama,1991,15
1,1,Alaska,1991,10
2,2,Arizona,1991,12
3,3,Arkansas,1991,15
4,4,California,1991,58
...,...,...,...,...
1345,1345,Virginia,2017,13
1346,1346,Washington,2017,43
1347,1347,West Virginia,2017,21
1348,1348,Wisconsin,2017,23


In [27]:
dfSum = pd.DataFrame(df.groupby('State')['Law_Total'].sum())
# dfSum.index.rename('state', inplace=True)
dfSum.head(10)

,Law_Total
State,
Alabama,335
Alaska,170
Arizona,325
Arkansas,337
California,2298
Colorado,456
Connecticut,1768
Delaware,685
Florida,564


In [10]:
query = db.select([StateGeoLoc]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [11]:
# move table to df
st = pd.DataFrame(ResultSet)
st.columns = ResultSet[0].keys()

# df_part = df_top.tail(250)
# df_part
st.head(10)

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California
5,CO,39.550051,-105.782067,Colorado
6,CT,41.603221,-73.087749,Connecticut
7,DC,38.905985,-77.033418,District of Columbia
8,DE,38.910832,-75.527670,Delaware
9,FL,27.664827,-81.515754,Florida


In [29]:
newdf = pd.merge(dfSum, st,  how='inner', left_on=['State'], right_on = ['name'])
                 
newdf.head(10)

,Law_Total,state,latitude,longitude,name
0,335,AL,32.318231,-86.902298,Alabama
1,170,AK,63.588753,-154.493062,Alaska
2,325,AZ,34.048928,-111.093731,Arizona
3,337,AR,35.201050,-91.831833,Arkansas
4,2298,CA,36.778261,-119.417932,California
5,456,CO,39.550051,-105.782067,Colorado
6,1768,CT,41.603221,-73.087749,Connecticut
7,685,DE,38.910832,-75.527670,Delaware
8,564,FL,27.664827,-81.515754,Florida
9,287,GA,32.157435,-82.907123,Georgia


In [32]:
# Generate return values geojson
geojson = {"type": "FeatureCollection", "features": []}

for _, row in newdf.iterrows():
    feature = {"type": "Feature", "geometry": {"type": "Point",
               "coordinates": [row['longitude'], row['latitude']]},
               "properties": {"lawTotal": row['Law_Total'],"stateAbbr": row['state'],
                             "stateName": row['name']
                             }}
    geojson['features'].append(feature)
    
# return geojson
    
    
    
print(json.dumps(geojson, indent=4))
print('Done!')

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -86.902298,
                    32.318231
                ]
            },
            "properties": {
                "lawTotal": 335,
                "stateAbbr": "AL",
                "stateName": "Alabama"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -154.493062,
                    63.588753
                ]
            },
            "properties": {
                "lawTotal": 170,
                "stateAbbr": "AK",
                "stateName": "Alaska"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -111.093731,
     

In [33]:
# Write json file for testing
with open('regulations.json', 'w') as json_file:
    json.dump(geojson, json_file)  

## Mass Shootings

In [35]:
query = db.select([MassShootings]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [36]:
# move table to df
ms = pd.DataFrame(ResultSet)
ms.columns = ResultSet[0].keys()

# df_part = df_top.tail(250)
# df_part
ms.head(10)

,MassShootingID,case,location,location_type,date,age_of_shooter,summary,fatalities,weapon_type,weapon_details,injured,Latitude,longitude,type,gender
0,0,Uvalde elementary school massacre,"Uvalde, Texas",School,5/24/2022,18,DETAILS PENDING,21,semiautomatic rifle,Smith & Wesson,5,29.115800,-99.471800,Mass,M
1,1,Buffalo supermarket massacre,"Buffalo, New York",workplace,5/14/2022,18,"Payton S. Gendron, 18, committed a racially mo...",10,semiautomatic rifle,Bushmaster XM-15 semiautomatic rifle,3,42.543500,-78.511000,Mass,M
2,2,Sacramento County church shooting,"Sacramento, California",Religious,2/28/2022,-,"""A man believed to be meeting his three childr...",4,-,-,0,38.601110,-121.418966,Mass,M
3,3,Oxford High School shooting,"Oxford, Michigan",School,11/30/2021,15,"Ethan Crumbley, a 15-year-old student at Oxfor...",4,semiautomatic handgun,Sig Sauer 9mm pistol,7,42.844108,-83.259928,Mass,M
4,4,San Jose VTA shooting,"San Jose, California",Workplace,5/26/2021,57,"Samuel Cassidy, 57, a Valley Transportation Au...",9,semiautomatic handguns,-,0,37.316097,-121.888533,Mass,M
5,5,FedEx warehouse shooting,"Indianapolis, Indiana",Workplace,4/15/2021,19,"Brandon Scott Hole, 19, opened fire around 11 ...",8,semiautomatic rifle,-,7,39.686630,-86.323130,Mass,M
6,6,Orange office complex shooting,"Orange, California",Workplace,3/31/2021,-,"Aminadab Gaxiola Gonzalez, 44, allegedly opene...",4,semiautomatic handgun,-,1,33.835420,-117.853790,Mass,M
7,7,Boulder supermarket shooting,"Boulder, Colorado",Workplace,3/22/2021,21,"Ahmad Al Aliwi Alissa, 21, carried out a mass ...",10,"semiautomatic rifle, semiautomatic handgun; ta...",Ruger AR-556; weapon was purchased six days be...,0,39.986961,-105.251168,Mass,M
8,8,Atlanta massage parlor shootings,"Atlanta, Georgia",Workplace,3/16/2021,21,"Robert Aaron Long, 21, killed eight people at ...",8,-,-,1,34.111653,-84.580376,Spree,M
9,9,Springfield convenience store shooting,"Springfield, Missouri",Workplace,3/16/2020,31,"Joaquin S. Roman, 31, went on a rampage culmin...",4,-,-,0,37.210432,-93.236860,Mass,M


In [40]:
# Generate return values geojson
geojson = {"type": "FeatureCollection", "features": []}

for _, row in ms.iterrows():
    feature = {"type": "Feature", "geometry": {"type": "Point",
               "coordinates": [row['longitude'], row['Latitude']]},
               "properties": {"case": row['case'],
                              "location": row['location'],
                              "location_type": row['location_type'],
                              "age_of_shooter": row['age_of_shooter'],
                              "fatalities": row['fatalities'],
                              "weapon_type": row['weapon_type'],
                              "injured": row['injured'],
                              "type": row['type'],
                              "date": row['date']
                             }}
    geojson['features'].append(feature)
    
# return geojson
    
    
    
print(json.dumps(geojson, indent=4))
print('Done!')

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -99.4718,
                    29.1158
                ]
            },
            "properties": {
                "case": "Uvalde elementary school massacre",
                "location": "Uvalde, Texas",
                "location_type": "School",
                "age_of_shooter": "18",
                "fatalities": 21,
                "weapon_type": "semiautomatic rifle",
                "injured": 5,
                "type": "Mass",
                "date": "5/24/2022"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -78.511,
                    42.5435
                ]
            },
            "properties": {
                "case": "Buffalo supermarket

In [41]:
# Write json file for testing
with open('massshootings.json', 'w') as json_file:
    json.dump(geojson, json_file)  